In [59]:
!{sys.executable} -m pip install numpy

In [79]:
import numpy as np
from numpy import zeros as zeros

def coefficients_to_string(values: [float]):
    result = ""
    for i, value in enumerate(values):
        if value == 0:
            continue

        if i != 0:
            result += " "
            result += "-" if value < 0 else "+"
            result += " "
            if abs(value) != 1:
                result += abs(value)
        else:
            if value == -1:
                result += "-"
            else:
                if value != 1:
                    result += str(value)

        result += f"x{i + 1}"

    return result

class Function:
    coefficients: [float]
    optimize_max: bool

    def __init__(self, coefficients: [float], optimize_max: bool = True):
        self.coefficients = np.array(coefficients, dtype=float)
        self.optimize_max = optimize_max

    def negate(self):
        self.coefficients *= -1
        self.optimize_max = not self.optimize_max

    def __str__(self):
        result = coefficients_to_string(self.coefficients)
        result += " -> "
        result += "max" if self.optimize_max else "min"

        return result

class Constraint:
    coefficients: [float]
    value: float
    sign: int

    def __init__(self, coefficients: [float], value: float, sign: int = 0):
        self.coefficients = np.array(coefficients, dtype=float)
        self.value = value
        self.sign = sign

    def sign_less(self):
        return self.sign < 0

    def sign_equals(self):
        return self.sign == 0

    def sign_greater(self):
        return self.sign > 0

    def set_sign_equals(self):
        self.sign = 0

    def sign_to_string(self):
        if self.sign_less():
            return "<="
        else:
            if self.sign == 0:
                return "="
            else:
                return ">="

    def negate(self):
        self.coefficients *= -1
        self.value *= -1
        self.sign *= -1

    def __str__(self):
        result = coefficients_to_string(self.coefficients)
        result += " " + self.sign_to_string()
        result += " " + str(self.value)

        return result

class LinearProgrammingTask:
    function: Function
    constraints: [Constraint]

    def __init__(self, function: Function, constraints: [Constraint]):
        self.function = function
        self.constraints = constraints

    def to_canonic_form(self):
        if not self.function.optimize_max:
            self.function.negate()

        free_variables_count = 0
        for constraint in self.constraints:
            if constraint.sign != 0:
                free_variables_count += 1
                if constraint.sign_greater():
                    constraint.negate()

        np.append(self.function.coefficients, zeros(free_variables_count))

        current_free_variables_count = 0
        for constraint in self.constraints:
            if constraint.sign == 0:
                np.append(constraint.coefficients, zeros(free_variables_count))
            else:
                np.append(constraint.coefficients, (zeros(current_free_variables_count)))
                np.append(constraint.coefficients, 1)
                np.append(constraint.coefficients, zeros(free_variables_count - current_free_variables_count - 1))
                current_free_variables_count += 1
                constraint.set_sign_equals()

            if constraint.value < 0:
                constraint.negate()

    def dimensions(self) -> (int, int):
        return len(self.function.coefficients), len(self.constraints)

    def __str__(self):
        result = str(self.function) + "\n"
        result += "\n".join(str(constraint) for constraint in self.constraints)

        return result

In [83]:
from math import inf

eps = 1e-10

def find_basic_feasible_solution(lpt: LinearProgrammingTask) -> [float]:
    n, m = lpt.dimensions()
    basic_solution = np.zeros(n).tolist()
    function = Function(basic_solution + [-1] * m)
    constraints = []
    for i, constraint in enumerate(lpt.constraints):
        coefficients = []
        coefficients.extend(constraint.coefficients.tolist())
        coefficients.extend(zeros(i).tolist())
        coefficients.append(1)
        coefficients.extend(zeros(m - i - 1).tolist())

        value = constraint.value
        constraints.append(Constraint(coefficients, value, 0))
        if value == 0:
            value = inf

        basic_solution.append(value)

    solution_coefficients = solve_lpt(LinearProgrammingTask(function, constraints), basic_solution).coefficients
    return solution_coefficients[:n]

def extract_basis(constraints: [Constraint], basic_feasible_solution: [float], n: int, m: int) -> [int]:
    basis = []
    for i, value in enumerate(basic_feasible_solution):
        if value != 0:
            basis.append(i)

    constraints.sort(reverse=True, key=lambda constraint: [constraint.coefficients[i] for i in basis])
    i = 0
    while True:
        if i >= m:
            break

        zero_coeffs_count = 0
        for value in constraints[i].coefficients:
            if value < eps:
                zero_coeffs_count += 1

        if zero_coeffs_count == len(constraints[i].coefficients):
            constraints.remove(constraints[i])
            i -= 1
            m -= 1
            continue

        basis_value = constraints[i].coefficients[basis[i]]
        constraints[i].coefficients /= basis_value
        constraints[i].value /= basis_value

        for j in range(m):
            if i == j:
                continue

            delta = constraints[j].coefficients[basis[i]]
            for k in range(len(constraints[j].coefficients)):
                constraints[j].coefficients[k] -= delta * constraints[i].coefficients[k]
            constraints[j].value -= delta * constraints[i].value

        constraints.sort(reverse=True, key=lambda c: [c.coefficients[i] for i in basis])
        i += 1

    return basis


def fill_table(function: Function, constraints: [Constraint], basis: [float], n, m) -> [[float]]:
    table = list(map(lambda constraint: [constraint.value] + constraint.coefficients.tolist(), constraints))
    function_row = zeros(n).tolist()
    for i in range(n - 1):
        if i not in basis:
            function_row[i + 1] += function.coefficients[i]
        else:
            for constraint in constraints:
                if constraint.coefficients[i] == 0:
                    continue
                coefficients = constraint.coefficients
                for j in range(n - 1):
                    if j == i:
                        continue
                    function_row[j + 1] -= coefficients[j] * function.coefficients[i] / coefficients[i]
                function_row[0] -= constraint.value * function.coefficients[i] / coefficients[i]
    table.append(function_row)

    return table

def sign(i: float) -> float:
    if i >= 0.0:
        return 1
    return -1

def find_better_basis(table: [[float]], basis: [float], n: int, m: int):
    while True:
        max_simplex_diff = 0
        lead_element_j = -1
        for j in range(1, n):
            if table[m][j] > max_simplex_diff:
                max_simplex_diff = table[m][j]
                lead_element_j = j
        if max_simplex_diff < eps:
            break

        min_fraction = inf
        if table[0][lead_element_j] != 0:
            if sign(table[0][0]) == sign(table[0][lead_element_j]):
                min_fraction = table[0][0] / table[0][lead_element_j]
        lead_element_i = 0
        for i in range(m):
            fraction = inf
            if table[i][lead_element_j] != 0:
                fraction = table[i][0] / table[i][lead_element_j]

            if sign(table[i][0]) == sign(table[i][lead_element_j]) and fraction < min_fraction:
                min_fraction = fraction
                lead_element_i = i

        lead_element = table[lead_element_i][lead_element_j]
        for j in range(n):
            table[lead_element_i][j] /= lead_element

        for i in range(m + 1):
            if i == lead_element_i:
                continue

            d = -table[i][lead_element_j]
            for j in range(n):
                table[i][j] += d * table[lead_element_i][j]

        basis[lead_element_i] = lead_element_j - 1

def build_table(lpt: LinearProgrammingTask, basic_feasible_solution: [float], n: int, m: int) -> ([[float]], [int]):
    basis = extract_basis(lpt.constraints, basic_feasible_solution, n, m)
    table = fill_table(lpt.function, lpt.constraints, basis, n, m)

    return table, basis

def simplex_method(lpt: LinearProgrammingTask, basic_feasible_solution: [float]) -> Constraint:
    n, m = lpt.dimensions()
    n += 1
    table, basis = build_table(lpt, basic_feasible_solution, n, m)
    find_better_basis(table, basis, n, len(lpt.constraints))

    coefficients = zeros(len(basic_feasible_solution)).tolist()
    n, m = lpt.dimensions()
    for i in range(m):
        coefficients[basis[i]] = table[i][0]

    value = -table[m][0]

    return Constraint(coefficients, value)

def solve_lpt(lpt: LinearProgrammingTask, basic_feasible_solution: [float] = None) -> Constraint:
    lpt.to_canonic_form()

    if not basic_feasible_solution:
        basic_feasible_solution = find_basic_feasible_solution(lpt)

    return simplex_method(lpt, basic_feasible_solution)

In [84]:
lpt = LinearProgrammingTask(
    function=Function(coefficients=[-1, -1, 0, -5]),
    constraints=[
        Constraint([1, 1, -1, 3], 1, 0),
        Constraint([1, -2, 3, -1], 1, 0),
        Constraint([5, -4, 7, 3], 5, 0)
    ]
)
answer = solve_lpt(lpt, None)
print(answer)

x1 = -1.0
